<a href="https://colab.research.google.com/github/Pranjalya/fake-news-twitter/blob/master/Fake_News_Countering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Spark

Installing dependencies (JAVA, Hadoop and Spark)

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q findspark
!pip install -q pyspark
!pip install -q tweet-preprocessor

Creating Spark session using PySpark

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Downloading fact-checking twitter data.

In [35]:
!curl 'https://storage.googleapis.com/kaggle-data-sets/689642%2F1213945%2Fcompressed%2FAllTweets_Dec2019_June2020.csv.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1591776960&Signature=lOPPi5k%2FlvyJT8uawwMzZsSZ1m%2B2XT2ti%2BMWBQzXcAj5zPbsMepoaGfQ2pRYai3aZEAc9R9R8Y8BtSYKP5fEfwpN1SyOaRtYcU%2BsqnK7xWVqc5cNYl3sBzOiYBZDOA1DnaopD213MF1EjNtdcg3tFzbZ9X%2FReXzEuHShkPtFjfWudnchNRNKkMpLw5twvLxFtyrWt4tMlTdLo7E35AdDedW%2FEoBC1dOGUvGrdg3%2Feh%2F6B0H7WM25i79%2FblRU5Ef7oU%2F5lfMUeNFfEpuxmOdvsq8jvlAxUAS5FXpKvFVvmBPawEBAnkp6p9LYyOg3unAaHLLGQVxQAQdFzTVtUt8o6Q%3D%3D' -H 'authority: storage.googleapis.com' -H 'upgrade-insecure-requests: 1' -H 'dnt: 1' -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36' -H 'sec-fetch-dest: document' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' -H 'sec-fetch-site: cross-site' -H 'sec-fetch-mode: navigate' -H 'sec-fetch-user: ?1' -H 'referer: https://www.kaggle.com/' -H 'accept-language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7' --compressed --output fact_checker.zip
!mkdir data
!unzip fact_checker.zip -d data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.1M  100 10.1M    0     0  28.8M      0 --:--:-- --:--:-- --:--:-- 28.9M
mkdir: cannot create directory ‘data’: File exists
Archive:  fact_checker.zip
  inflating: data/AllTweets_Dec2019_June2020.csv  


## Loading data

In [38]:
import pandas as pd
import numpy as np
import os
import re
# Tweet preprocessor - https://github.com/s/preprocessor
import preprocessor as tp
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
path = '/content/drive/My Drive/twitter_dataset/'
fact_checker_path = '/content/data/AllTweets_Dec2019_June2020.csv'
first = True

for f in tqdm(os.listdir(path)):
    if(first):
        df = pd.read_json(path + f, compression='gzip', lines=True)
        fact_checker_df = pd.read_csv(fact_checker_path)
        first = True
    else:
        df = pd.concat([df, pd.read_json(path + f, compression='gzip', lines=True)])
df.info()

  0%|          | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 90/90 [02:59<00:00,  1.99s/it]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15686 entries, 0 to 15685
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 15686 non-null  datetime64[ns, UTC]
 1   id                         15686 non-null  int64              
 2   id_str                     15686 non-null  int64              
 3   full_text                  15686 non-null  object             
 4   truncated                  15686 non-null  bool               
 5   display_text_range         15686 non-null  object             
 6   entities                   15686 non-null  object             
 7   source                     15686 non-null  object             
 8   in_reply_to_status_id      839 non-null    float64            
 9   in_reply_to_status_id_str  839 non-null    float64            
 10  in_reply_to_user_id        882 non-null    float64            
 11  in

In [40]:
fact_checker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85622 entries, 0 to 85621
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               85622 non-null  int64  
 1   conversation_id  85622 non-null  int64  
 2   created_at       85622 non-null  int64  
 3   date             85622 non-null  object 
 4   timezone         85622 non-null  object 
 5   place            4 non-null      object 
 6   tweet            85622 non-null  object 
 7   hashtags         85622 non-null  object 
 8   cashtags         85622 non-null  object 
 9   user_id          85622 non-null  int64  
 10  user_id_str      85622 non-null  int64  
 11  username         85622 non-null  object 
 12  name             85622 non-null  object 
 13  day              85622 non-null  int64  
 14  hour             85622 non-null  int64  
 15  link             85622 non-null  object 
 16  retweet          85622 non-null  bool   
 17  nlikes      

In [41]:
df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,withheld_scope,withheld_in_countries
0,2020-01-23 18:59:28+00:00,1220420803336048642,1220420803336048640,RT @ConflictsW: Roads in Wuhan on lockdown and...,False,"[0, 140]","{'hashtags': [{'text': 'China', 'indices': [99...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 12537532, 'id_str': '12537532', 'name':...",None,None,None,NaN,{'created_at': 'Wed Jan 22 19:44:32 +0000 2020...,False,7443,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23 18:59:28+00:00,1220420805189931009,1220420805189931008,RT @BNODesk: Here's a current list of the coro...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 624000012, 'id_str': '624000012', 'name...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:02:45 +0000 2020...,False,353,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-23 18:59:29+00:00,1220420806104289286,1220420806104289280,RT @ScottPresler: Is anyone else following the...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 39666288, 'id_str': '39666288', 'name':...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:32:48 +0000 2020...,False,1241,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-23 18:59:29+00:00,1220420808985718785,1220420808985718784,RT @JackPosobiec: Huge \n\nWas just told by a ...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 1080984630331957249, 'id_str': '1080984...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:22:14 +0000 2020...,False,1161,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-23 18:59:30+00:00,1220420812123115520,1220420812123115520,RT @inteldotwav: Helpful diagram by the WHO sh...,False,"[0, 123]","{'hashtags': [{'text': 'coronavirus', 'indices...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 43403644, 'id_str': '43403644', 'name':...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:52:04 +0000 2020...,False,141,0,False,False,en,"{'media': [{'id': 1220418936216412160, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
fact_checker_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Region
0,1267487446809313282,1267487446809313282,1591027530000,2020-06-01 16:05:30,UTC,NaN,"Aarti Lalchandani, principal of GVSM Medical C...",['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,1,16,https://twitter.com/AltNews/status/12674874468...,False,811,40,373,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
1,1267108282164019202,1267108282164019202,1590937130000,2020-05-31 14:58:50,UTC,NaN,Times Now played three videos during a show on...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,7,14,https://twitter.com/AltNews/status/12671082821...,False,278,9,92,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
2,1266732527227744256,1266732527227744256,1590847543000,2020-05-30 14:05:43,UTC,NaN,A video of an incident from Surat has been sha...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,14,https://twitter.com/AltNews/status/12667325272...,False,148,1,56,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
3,1266671072671391744,1266671072671391744,1590832891000,2020-05-30 10:01:31,UTC,NaN,A video showing poor condition of Mumbai's KEM...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,10,https://twitter.com/AltNews/status/12666710726...,False,157,5,64,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
4,1266653343767597057,1266653343767597057,1590828665000,2020-05-30 08:51:05,UTC,NaN,"Criticising journalists on COVID-19 reporting,...",['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,8,https://twitter.com/AltNews/status/12666533437...,False,439,25,196,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others


Let's take a look at the sentences of the fact-checker tweets, and see the term it uses to check the news.

In [53]:
fact_checker_df.iloc[6000:6005]

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Region
6000,1263425998717980675,1263425998717980675,1590059206000,2020-05-21 11:06:46,UTC,NaN,A video showing a crowded market in Pakistan's...,"['#lockdown', '#covid19', '#fakenews']",[],2392031700,2392031700,boomlive_in,BOOM Live,4,11,https://twitter.com/boomlive_in/status/1263425...,False,14,1,3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2392031700', 'username': 'boomli...",NaN,NaN,NaN,NaN,Others
6001,1263422145314484225,1263055756393189376,1590058287000,2020-05-21 10:51:27,UTC,NaN,https://www.boomlive.in/fake-news/video-from-...,[],[],2392031700,2392031700,boomlive_in,BOOM Live,4,10,https://twitter.com/boomlive_in/status/1263422...,False,1,0,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2392031700', 'username': 'boomli...",NaN,NaN,NaN,NaN,Others
6002,1263422118047309824,1239943348896428032,1590058280000,2020-05-21 10:51:20,UTC,NaN,https://www.boomlive.in/fake-news/video-from-...,[],[],2392031700,2392031700,boomlive_in,BOOM Live,4,10,https://twitter.com/boomlive_in/status/1263422...,False,1,0,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2392031700', 'username': 'boomli...",NaN,NaN,NaN,NaN,Others
6003,1263422038632361984,1263165564123398144,1590058261000,2020-05-21 10:51:01,UTC,NaN,https://www.boomlive.in/fake-news/video-from-...,[],[],2392031700,2392031700,boomlive_in,BOOM Live,4,10,https://twitter.com/boomlive_in/status/1263422...,False,1,0,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2392031700', 'username': 'boomli...",NaN,NaN,NaN,NaN,Others
6004,1263421878414094337,1263079795773956097,1590058223000,2020-05-21 10:50:23,UTC,NaN,https://www.boomlive.in/fake-news/video-from-...,[],[],2392031700,2392031700,boomlive_in,BOOM Live,4,10,https://twitter.com/boomlive_in/status/1263421...,False,2,1,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2392031700', 'username': 'boomli...",NaN,NaN,NaN,NaN,Others


At the end of each fact-checking tweet, there's a link to actual blog, on which the results of the claims are published. Keeping them in a seperate field, as they would get removed in preprocessing.

In [57]:
fact_checker_df['relay_url'] = fact_checker_df['tweet'].str.extract(
    pat="(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85622 entries, 0 to 85621
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               85622 non-null  int64  
 1   conversation_id  85622 non-null  int64  
 2   created_at       85622 non-null  int64  
 3   date             85622 non-null  object 
 4   timezone         85622 non-null  object 
 5   place            4 non-null      object 
 6   tweet            85622 non-null  object 
 7   hashtags         85622 non-null  object 
 8   cashtags         85622 non-null  object 
 9   user_id          85622 non-null  int64  
 10  user_id_str      85622 non-null  int64  
 11  username         85622 non-null  object 
 12  name             85622 non-null  object 
 13  day              85622 non-null  int64  
 14  hour             85622 non-null  int64  
 15  link             85622 non-null  object 
 16  retweet          85622 non-null  bool   
 17  nlikes      

## Data Preprocessing

Getting all the list of emoticons and other pictorials first.

In [0]:
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

# All emoticons
emoticons = emoticons_happy.union(emoticons_sad)

Performing basic preprocessings.

In [0]:
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)

    # Removing retweets and 
    tweet = re.sub(r'RT', '', tweet)
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)

    # Replace=ing consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)

    # Removing emoticons from tweet
    tweet = emoji_pattern.sub(r'', tweet)

    # Filtering using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []

    for w in word_tokens:
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    
    return ' '.join(filtered_tweet)

Cleaning all the tweets.

In [0]:
# Cleaning using tweet-preprocessor tool first
df['full_text'] = df['full_text'].apply(tp.clean)

# Performing some extra custom cleaning steps
df['full_text'] = df['full_text'].apply(clean_tweets)

# Performing cleaning too fact-checking tweets
fact_checker_df['tweet'] = fact_checker_df['tweet'].apply(tp.clean)
fact_checker_df['tweet'] = fact_checker_df['tweet'].apply(clean_tweets)

In [63]:
df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,withheld_scope,withheld_in_countries
0,2020-01-23 18:59:28+00:00,1220420803336048642,1220420803336048640,Roads Wuhan lockdown blocked police city quara...,False,"[0, 140]","{'hashtags': [{'text': 'China', 'indices': [99...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 12537532, 'id_str': '12537532', 'name':...",None,None,None,NaN,{'created_at': 'Wed Jan 22 19:44:32 +0000 2020...,False,7443,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23 18:59:28+00:00,1220420805189931009,1220420805189931008,Here 's current list coronavirus lockdowns Wuh...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 624000012, 'id_str': '624000012', 'name...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:02:45 +0000 2020...,False,353,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-23 18:59:29+00:00,1220420806104289286,1220420806104289280,Is anyone else following Coronavirus outbreak ...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 39666288, 'id_str': '39666288', 'name':...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:32:48 +0000 2020...,False,1241,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-23 18:59:29+00:00,1220420808985718785,1220420808985718784,Huge Was told Chinese reporter CCP authorities...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 1080984630331957249, 'id_str': '1080984...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:22:14 +0000 2020...,False,1161,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-23 18:59:30+00:00,1220420812123115520,1220420812123115520,Helpful diagram WHO showing various symptoms W...,False,"[0, 123]","{'hashtags': [{'text': 'coronavirus', 'indices...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 43403644, 'id_str': '43403644', 'name':...",None,None,None,NaN,{'created_at': 'Thu Jan 23 18:52:04 +0000 2020...,False,141,0,False,False,en,"{'media': [{'id': 1220418936216412160, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN,NaN


## Clustering Twitter data